In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from keras.models import load_model

/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# categories = {'Usable Directly Categorical':    2, 
#               'Usable Directly Numeric':        0, 
#               'Usable With Extraction':         1, 
#               'Unusable':                       3, 
#               'Context_Specific':               4, 
#               }

In [3]:
categories = {'Usable Directly Categorical':    0, 
              'Usable Directly Numeric':        1, 
              'Usable With Extraction':         2, 
              'Unusable':                       3
#               'Context_Specific':               4, 
              }

In [24]:
data = pd.read_csv('results/train_raw_4.csv')
labels = pd.read_csv('results/train_marginals_4.csv',header=None).values

In [25]:
labels.shape, data.shape

((8691, 4), (8691, 16))

In [27]:
for i in range(labels.shape[1]):
    data[str(i)] = labels[:,i]

In [28]:
data['y_act'] = data['y_act'].map(categories)
data_LSTM = pd.concat([data['Attribute_name'], data['sample_1'], data['sample_2'], data['sample_3'], data['sample_4'], data['sample_5']], axis =1)

In [29]:
data = data.rename(columns={'num of dist_val': 'num_of_dist_val'})
data['p_dist_val'] = data.num_of_dist_val / data.Total_val
data['p_nans'] = data['% nans']

In [30]:
def get_castability(dat):
    sample_values = dat['sample_1':'sample_5']
    #make sure the value you are avaluating is not nan
    castability = 0
    for values in sample_values:
        if pd.isnull(values) == False and values.__class__.__name__ == 'str':
            try:
                castability = str(type(eval(values)))
                castability = 1
                break
            except:
                #print('passing')
                pass
    return castability  

def get_extractability(dat, cast):
    sample_values = dat['sample_1':'sample_5']
    #make sure the value you are avaluating is not nan
    extractability = 0
    for values in sample_values:
        if pd.isnull(values) == False and values.__class__.__name__ == 'str' and cast == 0:
            try:
                #it will faile when you have no numbers or if you have two numbers seperated by space
                float(re.sub('[^0-9\. ]', ' ',values))
                #print('yes')
                extractability = 1
                break
            except:
                pass
    return extractability

def get_len(dat):
    sample_values = dat['sample_1':'sample_5']
    #make sure the value you are avaluating is not nan
    length = 0
    for values in sample_values:
        if pd.isnull(values) == False:
            length = len(str(values).split()) + length
    return length/5.0 

In [31]:
from tqdm import tqdm
castability = []
extractability = []
len_val = []
for i in tqdm(range(len(data))):
    castability.append(get_castability(data.loc[i]))
    extractability.append(get_extractability(data.loc[i], castability[i]))
    len_val.append(get_len(data.loc[i]))
data['castability'] = castability
data['extractability'] = extractability
data['len_val'] = len_val  

100%|██████████| 8691/8691 [00:10<00:00, 793.39it/s]


In [32]:
data.columns.tolist()

['Unnamed: 0',
 'y_act',
 'Reason',
 'Attribute_name',
 'Total_val',
 'num_of_dist_val',
 '% nans',
 'max_val',
 'mean',
 'min_val',
 'std_dev',
 'sample_1',
 'sample_2',
 'sample_3',
 'sample_4',
 'sample_5',
 '0',
 '1',
 '2',
 '3',
 'p_dist_val',
 'p_nans',
 'castability',
 'extractability',
 'len_val']

In [33]:
data.head()

,Unnamed: 0,y_act,Reason,Attribute_name,Total_val,num_of_dist_val,% nans,max_val,mean,min_val,...,sample_5,0,1,2,3,p_dist_val,p_nans,castability,extractability,len_val
0,0,3,l,StratificationCategory2,403984,1,0.803648,0.0,0.0,0.0,...,NaN,0.071483,0.071483,0.071483,0.785551,0.000002,0.803648,0,0,0.0
1,1,3,l,Stratification2,403984,1,0.803648,0.0,0.0,0.0,...,NaN,0.071483,0.071483,0.071483,0.785551,0.000002,0.803648,0,0,0.0
2,2,3,l,StratificationCategory3,403984,1,0.803648,0.0,0.0,0.0,...,NaN,0.071483,0.071483,0.071483,0.785551,0.000002,0.803648,0,0,0.0
3,3,3,l,Stratification3,403984,1,0.803648,0.0,0.0,0.0,...,NaN,0.071483,0.071483,0.071483,0.785551,0.000002,0.803648,0,0,0.0
4,4,2,c,GeoLocation,403984,55,0.006357,0.0,0.0,0.0,...,"(37.63864012300047, -120.99999953799971)",0.095930,0.027548,0.848974,0.027548,0.000136,0.006357,1,0,2.0


In [34]:
data.to_csv('processed_raw_4.csv')

## After prepossessing

In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from keras.models import load_model

In [2]:
data = pd.read_csv('processed_raw.csv')
labels = np.loadtxt('train_marginals.txt')

In [3]:
data.head()

,Unnamed: 0,Unnamed: 0.1,y_act,Reason,Attribute_name,Total_val,num_of_dist_val,% nans,max_val,mean,...,0,1,2,3,4,p_dist_val,p_nans,castability,extractability,len_val
0,0,0,3,l,StratificationCategory2,403984,1,80.364816,0.0,0.0,...,0.044124,0.044124,0.044124,0.823504,0.044124,0.000002,80.364816,0,0,0.0
1,1,1,3,l,Stratification2,403984,1,80.364816,0.0,0.0,...,0.044124,0.044124,0.044124,0.823504,0.044124,0.000002,80.364816,0,0,0.0
2,2,2,3,l,StratificationCategory3,403984,1,80.364816,0.0,0.0,...,0.044124,0.044124,0.044124,0.823504,0.044124,0.000002,80.364816,0,0,0.0
3,3,3,3,l,Stratification3,403984,1,80.364816,0.0,0.0,...,0.044124,0.044124,0.044124,0.823504,0.044124,0.000002,80.364816,0,0,0.0
4,4,4,2,c,GeoLocation,403984,55,0.635669,0.0,0.0,...,0.184907,0.040923,0.692323,0.040923,0.040923,0.000136,0.635669,1,0,2.0


In [4]:
def abs_limit(x):
    if abs(x) > 10000:
        return 10000*np.sign(x)
    return x

In [5]:
data = data.fillna(0)
data['scaled_mean'] = data['mean'].apply(abs_limit)
data['scaled_min_val'] = data['min_val'].apply(abs_limit)
data['scaled_max_val'] = data['max_val'].apply(abs_limit)
data['scaled_std_dev'] = data['std_dev'].apply(abs_limit)

In [6]:
column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev']
x = data[column_names_to_normalize].values
x = np.nan_to_num(x)
x_scaled = StandardScaler().fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data.index)
data[column_names_to_normalize] = df_temp

In [7]:
data.head()

,Unnamed: 0,Unnamed: 0.1,y_act,Reason,Attribute_name,Total_val,num_of_dist_val,% nans,max_val,mean,...,4,p_dist_val,p_nans,castability,extractability,len_val,scaled_mean,scaled_min_val,scaled_max_val,scaled_std_dev
0,0,0,3,l,StratificationCategory2,403984,1,80.364816,0.0,0.0,...,0.044124,0.000002,80.364816,0,0,0.0,-0.300157,0.038284,-0.442185,-0.366826
1,1,1,3,l,Stratification2,403984,1,80.364816,0.0,0.0,...,0.044124,0.000002,80.364816,0,0,0.0,-0.300157,0.038284,-0.442185,-0.366826
2,2,2,3,l,StratificationCategory3,403984,1,80.364816,0.0,0.0,...,0.044124,0.000002,80.364816,0,0,0.0,-0.300157,0.038284,-0.442185,-0.366826
3,3,3,3,l,Stratification3,403984,1,80.364816,0.0,0.0,...,0.044124,0.000002,80.364816,0,0,0.0,-0.300157,0.038284,-0.442185,-0.366826
4,4,4,2,c,GeoLocation,403984,55,0.635669,0.0,0.0,...,0.040923,0.000136,0.635669,1,0,2.0,-0.300157,0.038284,-0.442185,-0.366826


In [8]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'y_act', 'Reason', 'Attribute_name',
       'Total_val', 'num_of_dist_val', '% nans', 'max_val', 'mean', 'min_val',
       'std_dev', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5',
       '0', '1', '2', '3', '4', 'p_dist_val', 'p_nans', 'castability',
       'extractability', 'len_val', 'scaled_mean', 'scaled_min_val',
       'scaled_max_val', 'scaled_std_dev'],
      dtype='object')

In [9]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=100)
data_train.shape, data_test.shape

((8592, 31), (2149, 31))

In [10]:
x_columns =  ['Attribute_name',
                  'sample_1',
                  'sample_2',
                  'sample_3',
                  'sample_4',
                  'sample_5',
                  'p_nans',
                  'p_dist_val',
                  'scaled_max_val', 
                  'scaled_mean', 
                  'scaled_min_val',
                  'scaled_std_dev',
                  'castability',
                  'extractability',
                  'len_val']

y_columns = ['0', '1', '2', '3', '4']

X_train = data_train[x_columns].values
X_test = data_test[x_columns].values
y_train = data_train[y_columns].values.astype(float)
y_test = data_test[y_columns].values.astype(float)
y_act = data_test['y_act'].values.astype(int)
X_train.shape, X_test.shape, y_train.shape, y_test.shape, y_act.shape

((8592, 15), (2149, 15), (8592, 5), (2149, 5), (2149,))

In [12]:
sample_base = 'sample_'
# samples_train = []
# samples_test = []

i = 1
field_name = sample_base + str(i)
field_idx = x_columns.index(field_name)

samples1_train = X_train[:,field_idx].astype(str)
samples1_test = X_test[:,field_idx].astype(str)

In [14]:
field_name = 'Attribute_name'
atr_Xtrain = X_train[:,x_columns.index(field_name)].astype(str)
atr_Xtest = X_test[:,x_columns.index(field_name)].astype(str)

In [15]:
atr_Xtrain

array(['pixel612', 's2p2c1seed', 'pixel291', ..., 'q25Scala',
       'number_of_panels_s', 'tBodyAccJerk.max.X'], dtype='<U91')

In [16]:
# define network parameters
max_features = 128
maxlen = 128

In [18]:
def generate_padded_tokens(train_list, test_list, maxlen=maxlen):
    tokenizer = keras_text.Tokenizer(char_level = True)
    tokenizer.fit_on_texts(train_list)
    tokenized_train = tokenizer.texts_to_sequences(train_list)
    pad_train = keras_seq.pad_sequences(tokenized_train, maxlen)

    tokenized_test = tokenizer.texts_to_sequences(test_list)
    pad_test = keras_seq.pad_sequences(tokenized_test, maxlen)
    return tokenizer, pad_train, pad_test

In [19]:
atr_tokenizer, atr_pad_train, atr_pad_test = generate_padded_tokens(atr_Xtrain, atr_Xtest)

# sample_tokenizers = list()
# sample_pad_trains = list()
# sample_pad_tests = list()
# for i in range(5):
#     tokenizer, pad_train, pad_test = generate_padded_tokens(samples_train[i], samples_test[i])
#     sample_tokenizers.append(tokenizer)
#     sample_pad_trains.append(pad_train)
#     sample_pad_tests.append(pad_test)

sam_tokenizer, sam_pad_train, sam_pad_test = generate_padded_tokens(samples1_train, samples1_test)

In [20]:
X_train_left = X_train[:,6:]
X_test_left = X_test[:,6:]
X_train_left.shape, X_test_left.shape

((8592, 9), (2149, 9))

In [21]:
# X_train_pro = np.concatenate([atr_pad_train]+sample_pad_trains+[X_train_left],axis=1)
# X_test_pro = np.concatenate([atr_pad_test]+sample_pad_tests+[X_test_left],axis=1)
# X_train_pro.shape, X_test_pro.shape

In [22]:
X_t = atr_pad_train
X_t1 = sam_pad_train
structured_data_train = X_train_left
X_te = atr_pad_test
X_te1 = sam_pad_test
structured_data_test = X_test_left
X_t.shape, X_t1.shape, structured_data_train.shape

((8592, 128), (8592, 128), (8592, 9))

In [24]:
def build_model(numfilters,embed_size, tokenizer1, tokenizer2):
#     embed_size = 512
    inp = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer1.word_counts)+1, output_dim = embed_size)(inp)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []

    x = prefilt_x
    for i in range(2):
        x = Conv1D(numfilters*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    out_conv += [GlobalMaxPool1D()(x)]
    xy = concatenate(out_conv, axis = -1)  
    print(xy.shape)
    #########################################################
    inp1 = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer2.word_counts)+1, output_dim = embed_size)(inp1)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []

    x = prefilt_x
    for i in range(2):
        x = Conv1D(numfilters*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    out_conv += [GlobalMaxPool1D()(x)]
    x1 = concatenate(out_conv, axis = -1)
    print(x1.shape)
#     x1 = Flatten()(out_conv)
    #########################################################
#     inp2 = Input(shape=(None, ))
#     x = Embedding(input_dim = len(tokenizer2.word_counts)+1, output_dim = embed_size)(inp1)
# #     prefilt_x = Dropout(0.25)(x)
#     out_conv = []
#     # dilation rate lets us use ngrams and skip grams to process 
#     for dilation_rate in dilation_rates:
# #         x = prefilt_x
#         for i in range(2):
#             if dilation_rate>0:
#                 x = Conv1D(16*2**(i), 
#                            kernel_size = 3, 
#                            dilation_rate = dilation_rate,
#                           activation = 'relu',
#                           name = 'ngram_{}_cnn2_{}'.format(dilation_rate, i)
#                           )(x)
#             else:
#                 x = Conv1D(16*2**(i), 
#                            kernel_size = 1,
#                           activation = 'relu',
#                           name = 'word_fcl2_{}'.format(i))(x)
# #         out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
#         out_conv += [GlobalMaxPool1D()(x)]
#     x2 = concatenate(out_conv, axis = -1)
#     print(x2.shape)
#     x1 = Flatten()(out_conv)
#     #########################################################
    Str_input = Input(shape=(9,))
    layersfin = keras.layers.concatenate([xy,x1,Str_input])
    print(layersfin.shape)
    x = Dense(1000, activation='relu')(layersfin)
    x = Dropout(0.1)(x)
    x = Dense(1000, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(inputs=[inp,inp1,Str_input], outputs=[x])
#     model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# model = build_model()
# model.summary()

In [25]:
dummy_model = build_model(10,5,atr_tokenizer, sam_tokenizer)
dummy_model.metrics_names
dummy_model.summary()

(?, 40)
(?, 40)
(?, 89)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 5)      340         input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 5)      545         input_2[0][0]                    
_____________________________________________________________________________________

In [21]:
# n_filters_grid = [16,32,64,128]
# embed_size = [32,64,128,512]


n_filters_grid = [64]
embed_size = [128]

batch_size = 32 # large enough that some other labels come in
epochs = 50

k = 5
kf = KFold(n_splits=k)
# n_filters_grid = [32]
# embed_size = [256]

avgsc_lst,avgsc_train_lst = [],[]
avgsc,avgsc_train = 0,0
bestne,bestmd = 0,0
i=0
for train_index, test_index in kf.split(X_t):
    print('\n')
    X_train_cur, X_test_cur = X_t[train_index], X_t[test_index]
    X_train_cur1, X_test_cur1 = X_t1[train_index], X_t1[test_index]    
    y_train_cur, y_test_cur = y_train[train_index], y_train[test_index]
    structured_data_train_cur, structured_data_test_cur = structured_data_train[train_index],structured_data_train[test_index]
    
    X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=100)
    X_train_train1, X_val1 = train_test_split(X_train_cur1, test_size=0.25,random_state=100)    
    structured_data_train_train,structured_data_val = train_test_split(structured_data_train_cur, test_size=0.25,random_state=100)
    
    print(y_train_train.shape)
    print(y_val.shape)
    print(structured_data_train_train.shape)
    print(structured_data_val.shape)
    
    bestPerformingModel = build_model(10,5,atr_tokenizer, sam_tokenizer)
    bestscore = 0
    for ne in n_filters_grid:
        for md in embed_size:
            file_path= 'best_weights'+str(ne)+str(md)+'.h5'
            checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
            early = EarlyStopping(monitor="val_acc", mode="max", patience=20)
            callbacks_list = [checkpoint, early] #early            
            
#             clf = RandomForestClassifier(n_estimators=ne,max_depth=md)
            clf = build_model(ne,md,atr_tokenizer, sam_tokenizer)
            history = clf.fit([X_train_train,X_train_train1,structured_data_train_train],
                              y_train_train,
                              validation_data=([X_val,X_val1,structured_data_val], y_val),
                              batch_size=batch_size, 
                              epochs=epochs, 
                              shuffle = True, 
                              callbacks=callbacks_list)
        
            clf.load_weights('best_weights'+str(ne)+str(md)+'.h5')
            sc = clf.evaluate([X_val,X_val1,structured_data_val],y_val)
            print(sc)
        
            if bestscore < sc:
                bestscore = sc
                bestPerformingModel = clf
                bestne,bestmd = ne,md
                print(bestPerformingModel)
    
#     bestPerformingModel.load_weights('best_weights.h5')
    bestPerformingModel.load_weights('best_weights'+str(bestne)+str(bestmd)+'.h5')
    loss, bscr_train = \
        bestPerformingModel.evaluate([X_train_train,X_train_train1,structured_data_train_train],y_train_train)
    print(loss, bscr_train)
    loss, bscr = \
        bestPerformingModel.evaluate([X_test_cur,X_test_cur1,structured_data_test_cur],y_test_cur)
    print(loss, bscr)
    
    avgsc_lst.append(bscr)
    avgsc_train_lst.append(bscr_train)      
    
    avgsc = avgsc + bscr
    avgsc_train = avgsc_train + bscr_train
    print('The training accuracy is:')
    print(bscr_train)
    print('The test accuracy is:')    
    print(bscr)
    print('\n')
    i=i+1


print(avgsc_train/k)
print(avgsc/k)



(5154, 5)
(1719, 5)
(5154, 9)
(1719, 9)
(?, 40)
(?, 40)
(?, 89)
(?, 256)
(?, 256)
(?, 521)
Train on 5154 samples, validate on 1719 samples
Epoch 1/50
5154/5154 [==============================] - 14s 3ms/step - loss: 0.0523 - acc: 0.6434 - val_loss: 0.0523 - val_acc: 0.6457

Epoch 00001: val_acc improved from -inf to 0.64572, saving model to best_weights64128.h5
Epoch 2/50
5154/5154 [==============================] - 13s 2ms/step - loss: 0.0496 - acc: 0.6903 - val_loss: 0.0429 - val_acc: 0.7504

Epoch 00002: val_acc improved from 0.64572 to 0.75044, saving model to best_weights64128.h5
Epoch 3/50
5154/5154 [==============================] - 13s 2ms/step - loss: 0.0349 - acc: 0.7253 - val_loss: 0.0127 - val_acc: 0.7958

Epoch 00003: val_acc improved from 0.75044 to 0.79581, saving model to best_weights64128.h5
Epoch 4/50
5154/5154 [==============================] - 13s 3ms/step - loss: 0.0127 - acc: 0.7918 - val_loss: 0.0102 - val_acc: 0.8133

Epoch 00004: val_acc improved from 0.79581

KeyboardInterrupt: 

In [103]:
X_t.shape, X_t1.shape, structured_data_train.shape, y_train.shape

((8592, 512), (8592, 512), (8592, 9), (8592, 1))

In [104]:

bestone = load_model('best_weights.h5')
print('---')
loss, acc = bestone.evaluate([X_t,X_t1,structured_data_train],to_categorical(y_train),verbose=1)
print(loss, acc)
loss, acc = bestone.evaluate([X_te,X_te1,structured_data_test],to_categorical(y_test),verbose=1)
print(loss, acc)


---
8592/8592 [==============================] - 13s 2ms/step
6.157857602757004 0.37639664804469275
2149/2149 [==============================] - 3s 1ms/step
5.93661403101286 0.37971149373187635


In [118]:
X_t.shape

(8592, 512)

In [120]:
X_t1

array([[ 0,  0,  0, ...,  0,  0, 31],
       [ 0,  0,  0, ...,  0,  0, 18],
       [ 0,  0,  0, ..., 23, 26, 31],
       ...,
       [ 0,  0,  0, ...,  2,  4,  9],
       [ 0,  0,  0, ...,  0,  0, 31],
       [ 0,  0,  0, ..., 36, 38, 34]], dtype=int32)

In [121]:
structured_data_train

,p_nans,scaled_max_val,scaled_mean,scaled_min_val,p_dist_val,scaled_std_dev,castability,extractability,len_val
0,0.000000,-0.367285,-0.274532,0.038284,0.004267,-0.336663,1.0,0.0,1.0
1,5.220381,-0.442185,-0.300157,0.038284,0.018756,-0.366826,1.0,0.0,1.0
2,0.000000,-0.367285,-0.261691,0.038284,0.004267,-0.336455,1.0,0.0,1.0
3,3.750000,-0.525882,-3.786845,-4.596337,0.960294,2.994287,1.0,0.0,1.0
4,0.000000,2.495087,3.186532,0.052652,0.996703,2.994287,1.0,0.0,1.0
5,100.000000,-0.442185,-0.300157,0.038284,0.000000,-0.366826,0.0,0.0,0.0
6,23.371887,-0.438660,-0.299104,0.038284,0.000938,-0.366274,1.0,0.0,0.8
7,0.000000,1.343201,-0.283824,0.038284,0.080252,-0.339652,1.0,0.0,1.0
8,0.000000,-0.367285,-0.294500,0.038284,0.025400,-0.350244,1.0,0.0,1.0
9,0.000000,-0.075026,-0.299079,-0.499332,0.124324,-0.315371,1.0,0.0,1.0
